## Loading Libraries and Extracting Dataset

In [ ]:
#Basic
import os
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import numpy as np
import pandas as pd
import zipfile

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

datasetzipfile = "datasets.zip"

# Check if the zip file exists
if os.path.exists(datasetzipfile):
    # Open the zip file in read mode
    with zipfile.ZipFile(datasetzipfile, 'r') as zip_ref:
        # Iterate through each file in the zip archive
        for file_info in zip_ref.infolist():
            # Check if the file already exists in the current directory
            if not os.path.exists(file_info.filename):
                zip_ref.extract(file_info)
else:
    print(f"{datasetzipfile} does not exist.")

train_path_dog = "datasets/train/dog"
train_path_cat = "datasets/train/cat"
valid_path_dog = "datasets/val/dog"
valid_path_cat = "datasets/val/cat"

test_path = "datasets/test"

#Count the Data Provided
def count_files_in_directory(path):
    try:
        # List all entries in the specified directory
        all_entries = os.listdir(path)
        
        # Count only the files (exclude directories)
        total_files = sum(1 for entry in all_entries if os.path.isfile(os.path.join(path, entry)))
        
        return total_files
    except Exception as e:
        return str(e)

# Specify the directory path
print("Total number of dog images in training data :", count_files_in_directory(train_path_dog))
print("Total number of cat images in training data :", count_files_in_directory(train_path_cat))
print()
print("Total number of dog images in validation data :", count_files_in_directory(valid_path_dog))
print("Total number of cat images in validation data :", count_files_in_directory(valid_path_cat))
print()
print("Total number of unknown images in test data :", count_files_in_directory(test_path))

## Data Exploration

In [ ]:
#Visualise Images of Dogs from Train Dataset

plt.figure(figsize=(20,20))
plt.subplots_adjust(hspace=0.4)

# Initialize a counter for the number of images plotted
count = 0

for i in range(10):
    filename = f'{train_path_dog}/dog.{i}.jpg'
    
    # Check if the file exists
    if os.path.exists(filename):
        plt.subplot(1, 10, count + 1)  # Create a subplot for the existing image
        image = imread(filename)
        plt.imshow(image)
        plt.title('Dog' + str(i), fontsize=12)
        plt.axis('off')
        
        count += 1  # Increment the counter for each plotted image

# Adjust the layout to accommodate the images
plt.show()

In [ ]:
#Visualise Images of Cats from Train Dataset

plt.figure(figsize=(20,20))
plt.subplots_adjust(hspace=0.4)

# Initialize a counter for the number of images plotted
count = 0

for i in range(10):
    filename = f'{train_path_cat}/cat.{i}.jpg'
    
    # Check if the file exists
    if os.path.exists(filename):
        plt.subplot(1, 10, count + 1)  # Create a subplot for the existing image
        image = imread(filename)
        plt.imshow(image)
        plt.title('Cat' + str(i), fontsize=12)
        plt.axis('off')
        
        count += 1  # Increment the counter for each plotted image

# Adjust the layout to accommodate the images
plt.show()

## MobileNetV2 Model

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image

# Paths to training and validation directories
train_path = "datasets/train"
valid_path = "datasets/val"

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Data generators for loading images from directories
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Function to create the model
"""Build and compile the MobileNetV2 model."""
# Load MobileNetV3Large with ImageNet weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Model Summary")
print("--------------")
print("CNN Model: MobileNetV2")
print(model.summary())

# Callbacks for learning rate reduction and early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

print()
print("Data generators are created.")
print("Early stopping and learning rate reduction callbacks are set.")
print()

# Save the model architecture as an image
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

# Display the model architecture
Image.open('model_architecture.png')

## Model Training

In [ ]:
# Set the number of runs
n_runs = 5
val_acc_results = []
val_loss_results = []

for run in range(n_runs):
    
    # Train the model
    history = model.fit(train_generator, 
                        validation_data=valid_generator,
                        epochs=25,
                        callbacks=[early_stop, reduce_lr])
    
    # Evaluate and store the validation accuracy
    val_loss, val_acc = model.evaluate(valid_generator)
    val_acc_results.append(val_acc)
    val_loss_results.append(val_loss)

    print("Run " + str(run+1) + " Results:")
    print(f"Validation accuracy: {val_acc:.4f}")
    print(f"Validation loss: {val_loss:.4f}")
    print()

## Average Accuracy and Loss

In [ ]:
# Calculate and print average results
average_val_acc = np.mean(val_acc_results)
average_val_loss = np.mean(val_loss_results)

print("Average Results:")
print(f"Average Validation Accuracy: {average_val_acc:.4f}")
print(f"Average Validation Loss: {average_val_loss:.4f}")

## Submission CSV

In [ ]:
# Initialize an empty list to hold the results
results = []

# Loop through each file in the test directory
for img_file in os.listdir(test_path):
    # Load and preprocess the image
    img_path = os.path.join(test_path, img_file)
    img = image.load_img(img_path, target_size=(224, 224))  # Size should match input size of the model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to match training preprocessing

    # Make a prediction
    prediction = model.predict(img_array)
    # If prediction >= 0.5, it's a dog (1); otherwise, it's a cat (0)
    prediction_label = 1 if prediction >= 0.5 else 0
    
    # Append the filename and prediction to the results list
    results.append([os.path.splitext(img_file)[0], prediction_label])

# Create a DataFrame for the results
df = pd.DataFrame(results, columns=['id', 'label'])

# Save the results to a CSV file
df.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")

## Extracting CIFAR-10 Dataset

In [ ]:
import tarfile
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os

# Path to the CIFAR-10 tar file
tar_file_path = 'cifar-10-python.tar'

# Extract the tar file
with tarfile.open(tar_file_path, 'r:') as tar:
    tar.extractall()
    print("Extraction completed.")

# Path to the extracted CIFAR-10 data
cifar10_data_path = './cifar-10-batches-py/'

# Function to load a batch of CIFAR-10 data
def load_cifar_batch(filename):
    with open(filename, 'rb') as file:
        batch = pickle.load(file, encoding='latin1')
        images = batch['data']
        labels = batch['labels']
        # CIFAR-10 images are 32x32 with 3 channels
        images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
        return images, labels

# Load a batch of CIFAR-10 data for viewing
batch_file = os.path.join(cifar10_data_path, 'data_batch_1')
images, labels = load_cifar_batch(batch_file)

# CIFAR-10 classes
cifar10_classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]

# Display some sample images with their labels
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.title(cifar10_classes[labels[i]])
    plt.axis('off')
plt.show()


## Modified MobileNetV2 for CIFAR-10 Dataset

In [ ]:
import tarfile
import pickle
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

# Path to CIFAR-10 tar file
tar_file_path = 'cifar-10-python.tar'

# Extract the tar file
with tarfile.open(tar_file_path, 'r:') as tar:
    tar.extractall()
    print("Extraction completed.")

# Path to extracted CIFAR-10 data
cifar10_data_path = './cifar-10-batches-py/'

# Load CIFAR-10 batches
def load_cifar_data(data_path):
    images, labels = [], []
    for i in range(1, 6):  # CIFAR-10 has 5 training batches
        with open(f"{data_path}data_batch_{i}", 'rb') as file:
            batch = pickle.load(file, encoding='latin1')
            images.append(batch['data'])
            labels.extend(batch['labels'])
    images = np.concatenate(images)
    images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape to (32, 32, 3)
    return images, np.array(labels)

# Load training and test data
X_train, y_train = load_cifar_data(cifar10_data_path)
with open(f"{cifar10_data_path}test_batch", 'rb') as file:
    test_batch = pickle.load(file, encoding='latin1')
    X_test = test_batch['data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    y_test = np.array(test_batch['labels'])

# Normalize pixel values
X_train, X_test = X_train / 255.0, X_test / 255.0

# Convert labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Build MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for learning rate reduction and early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop, reduce_lr]
)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")


In [ ]:
import tarfile
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Path to CIFAR-10 tar file
tar_file_path = 'cifar-10-python.tar'

# Extract the tar file
with tarfile.open(tar_file_path, 'r:') as tar:
    tar.extractall()
    print("Extraction completed.")

# Path to extracted CIFAR-10 data
cifar10_data_path = './cifar-10-batches-py/'

# Load CIFAR-10 batches
def load_cifar_data(data_path):
    images, labels = [], []
    for i in range(1, 6):  # CIFAR-10 has 5 training batches
        with open(f"{data_path}data_batch_{i}", 'rb') as file:
            batch = pickle.load(file, encoding='latin1')
            images.append(batch['data'])
            labels.extend(batch['labels'])
    images = np.concatenate(images)
    images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape to (32, 32, 3)
    return images, np.array(labels)

# Load training and test data
X_train, y_train = load_cifar_data(cifar10_data_path)
with open(f"{cifar10_data_path}test_batch", 'rb') as file:
    test_batch = pickle.load(file, encoding='latin1')
    X_test = test_batch['data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    y_test = np.array(test_batch['labels'])

# Normalize pixel values
X_train, X_test = X_train / 255.0, X_test / 255.0

# Convert labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Resize images to a higher resolution for MobileNetV2
X_train = tf.image.resize(X_train, (96, 96))
X_val = tf.image.resize(X_val, (96, 96))
X_test = tf.image.resize(X_test, (96, 96))

# Data Augmentation
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.1
)
datagen.fit(X_train)

# Build MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base_model.trainable = True

# Fine-tune by freezing early layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Model architecture
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

# Use label smoothing in loss
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-5),
              loss=loss, metrics=['accuracy'])

# Callbacks for learning rate reduction and early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")
